<a href="https://colab.research.google.com/github/Kimp25/Clasificador-de-gastos/blob/main/Prueba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import os
from google.colab import files

In [32]:

# Clase para predecir categorías de gastos
class ExpensePredictor:
    def __init__(self, model_path, count_vect_path, tfidf_path):
        self.clf = self.load_pickle(model_path)
        self.count_vect = self.load_pickle(count_vect_path)
        self.tfidf_transformer = self.load_pickle(tfidf_path)

    @staticmethod
    def load_pickle(filepath):
        """Carga un archivo pickle."""
        with open(file
                  path, 'rb') as f:
            return pickle.load(f)

    def predecir_categoria(self, descripcion):
        """Predice la categoría de un gasto dado su descripción."""
        X_new_counts = self.count_vect.transform([descripcion])
        X_new_tfidf = self.tfidf_transformer.transform(X_new_counts)
        return self.clf.predict(X_new_tfidf)[0]

    def procesar_csv(self, df, columna_descripcion):
        """Añade predicciones al DataFrame."""
        df['categoria_predicha'] = df[columna_descripcion].apply(self.predecir_categoria)
        return df

    @staticmethod
    def generar_grafico(df):
        """Crea un gráfico circular basado en las veces que aparece una categoría"""
        conteo_categorias = Counter(df['categoria_predicha'])
        plt.figure(figsize=(10, 8))
        plt.pie(conteo_categorias.values(),
                labels=conteo_categorias.keys(),
                autopct='%1.1f%%',
                startangle=90)
        plt.title("Distribución de Categorías de Gastos")
        plt.legend(conteo_categorias.keys(), title="Categorías", loc="upper right", bbox_to_anchor=(1, 0, 0.5, 1))
        plt.tight_layout()
        plt.show()

def main():
    # Configuración de rutas
    model_path = '/content/modelo_gastos (3).pkl'
    count_vect_path = '/content/count_vect (3).pkl'
    tfidf_path = '/content/tfidf_transformer (3).pkl'

    predictor = ExpensePredictor(model_path, count_vect_path, tfidf_path)

    while True:
        print("\n=== CLASIFICADOR DE GASTOS ===")
        print("1. Clasificar gasto individual")
        print("2. Clasificar desde archivo CSV")
        print("3. Salir")

        opcion = input("Selecciona una opción (1-3): ")

        if opcion == "1":
            while True:
                descripcion = input("\nDescripción del gasto (o 'salir' para terminar): ").strip()
                if descripcion.lower() == 'salir':
                    break
                categoria = predictor.predecir_categoria(descripcion)
                print(f"Categoría predicha: {categoria}")

        elif opcion == "2":
            print("Sube tu archivo CSV ...")
            uploaded = files.upload()
            filename = next(iter(uploaded))
            df = pd.read_csv(filename, encoding='latin-1')

            # Procesar y clasificar
            columna_descripcion = df.columns[0]  # Usar la primera columna del archivo CSV
            resultados = predictor.procesar_csv(df, columna_descripcion)

            # Guarda y descarga los resultados
            resultados.to_csv('resultados_clasificacion.csv', index=False)
            files.download('resultados_clasificacion.csv')
            print("\nArchivo descargado exitosamente.")

            # Genera el gráfico
            predictor.generar_grafico(resultados)

        elif opcion == "3":
            print("¡Hasta luego!")
            break
        else:
            print("Opción no válida. Por favor, selecciona 1, 2 o 3.")

if __name__ == "__main__":
    main()



=== CLASIFICADOR DE GASTOS ===
1. Clasificar gasto individual
2. Clasificar desde archivo CSV
3. Salir
Selecciona una opción (1-3): 1

Descripción del gasto (o 'salir' para terminar): Farmacia
Categoría predicha: Restaurante

Descripción del gasto (o 'salir' para terminar): Pizza
Categoría predicha: Restaurante

Descripción del gasto (o 'salir' para terminar): Integral
Categoría predicha: Salud

Descripción del gasto (o 'salir' para terminar): Internet
Categoría predicha: Entretenimiento

Descripción del gasto (o 'salir' para terminar): salir

=== CLASIFICADOR DE GASTOS ===
1. Clasificar gasto individual
2. Clasificar desde archivo CSV
3. Salir
Selecciona una opción (1-3): 3
¡Hasta luego!
